## Adding Filtering
**django-rest-framework** has support for **using various filter backends**. We can enable these in all resources by adding them to the DefaultsMixin

We configure the **SearchFilter** by adding a **search_fields** attribute to each ViewSet. We configure the **OrderingFilter** by adding a list of fields, which can be used for ordering the **ordering_fields**.

* [http://localhost:8080/api/tasks/?search=foo]
* [http://localhost:8080/api/tasks/?search=first] 

In [3]:
%%writefile scrum/board/views.py

from django.contrib.auth import get_user_model

from rest_framework import authentication, permissions, viewsets

from .models import Sprint, Task
from .serializers import SprintSerializer, TaskSerializer, UserSerializer

User = get_user_model()

class DefaultsMixin(object):
    """Default settings for view authentication, permissions, filtering and pagination."""
    
    authentication_classes = (
        authentication.BasicAuthentication,
        authentication.TokenAuthentication,
    )
    permission_classes = (
        permissions.IsAuthenticated,
    )
    paginate_by = 25
    paginate_by_param = 'page_size'
    max_paginate_by = 100
    filter_backends = (
        filters.DjangoFilterBackend,
        filters.SearchFilter,
        filters.OrderingFilter,
    )

class SprintViewSet(viewsets.ModelViewSet):
    """API endpoint for listing and creating sprints."""

    queryset = Sprint.objects.order_by('end')
    serializer_class = SprintSerializer
    search_fields = ('name', )
    ordering_fields = ('end', 'name', )

class TaskViewSet(DefaultsMixin, viewsets.ModelViewSet):
    """API endpoint for listing and creating tasks."""
    
    queryset = Task.objects.all()
    serializer_class = TaskSerializer
    search_fields = ('name', 'description', )
    ordering_fields = ('name', 'order', 'started', 'due', 'completed', )
        
class UserViewSet(DefaultsMixin, viewsets.ReadOnlyModelViewSet):
    """API endpoint for listing users."""
    
    lookup_field = User.USERNAME_FIELD
    lookup_url_kwarg = User.USERNAME_FIELD
    queryset = User.objects.order_by(User.USERNAME_FIELD)
    serializer_class = UserSerializer
    search_fields = (User.USERNAME_FIELD, )

Overwriting scrum/board/views.py


To handle additional filtering of the task, we can make use of the **DjangoFilterBackend**. This requires defining a **filter_class** on the TaskViewSet. The filter_class attribute should be a **subclass of django_filters.FilterSet**.

### 1. TaskFilter
Each field defined in **TaskFilter** will **translate into a query parameter, which the client can use to filter the result set.**

### 2. NullFilter
a task that isn’t currently assigned a sprint would be considered a backlog task. 

* [http://localhost:8080/api/tasks/?backlog=True]

### 3. Filter using username
The rest of the API uses the username as a unique identifier.

* [http://localhost:8080/api/tasks/?assigned=demo]

### 4. Filter sprints that haven't ended yet
** 정상적으로 안된다 -0-;;;**

use SprintFilter, **end_min is gte, end_max is lte**. These can be combined to limit sprints to a given date range.

* (2015-11-01 보다 크거나 같은 sprint)[http://localhost:8080/api/sprints/?end_min=2015-11-01]
* (2015-11-01 보다 작거나 같은 sprint)[http://localhost:8080/api/sprints/?end_max=2015-11-01]

In [11]:
%%writefile scrum/board/forms.py

import django_filters

from django.contrib.auth import get_user_model

from .models import Task, Sprint

User = get_user_model()

class NullFilter(django_filters.BooleanFilter):
    """Filter on a field set as null or not."""

    def filter(self, qs, value):
        if value is not None:
            return qs.filter(**{'%s__isnull' % self.name: value})
        return qs
    
class TaskFilter(django_filters.FilterSet):
    
    backlog = NullFilter(name='sprint')
    
    class Meta:
        model = Task
        fields = ('sprint', 'status', 'assigned', 'backlog')
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.filters['assigned'].extra.update(
            {'to_field_name': User.USERNAME_FIELD})
        
class SprintFilter(django_filters.FilterSet):

    end_min = django_filters.DateFilter(name='end', lookup_type='gte')
    end_max = django_filters.DateFilter(name='end', lookup_type='lte')

    class Meta:
        model = Sprint
        fields = ('end_min', 'end_max', )

Overwriting scrum/board/forms.py


In [12]:
%%writefile scrum/board/views.py

from django.contrib.auth import get_user_model

from rest_framework import authentication, permissions, viewsets, filters

from .forms import TaskFilter, SprintFilter
from .models import Sprint, Task
from .serializers import SprintSerializer, TaskSerializer, UserSerializer

User = get_user_model()

class DefaultsMixin(object):
    """Default settings for view authentication, permissions, filtering and pagination."""
    
    authentication_classes = (
        authentication.BasicAuthentication,
        authentication.TokenAuthentication,
    )
    permission_classes = (
        permissions.IsAuthenticated,
    )
    paginate_by = 25
    paginate_by_param = 'page_size'
    max_paginate_by = 100
    filter_backends = (
        filters.DjangoFilterBackend,
        filters.SearchFilter,
        filters.OrderingFilter,
    )

class SprintViewSet(viewsets.ModelViewSet):
    """API endpoint for listing and creating sprints."""

    queryset = Sprint.objects.order_by('end')
    serializer_class = SprintSerializer
    filter_class = SprintFilter
    search_fields = ('name', )
    ordering_fields = ('end', 'name', )

class TaskViewSet(DefaultsMixin, viewsets.ModelViewSet):
    """API endpoint for listing and creating tasks."""
    
    queryset = Task.objects.all()
    serializer_class = TaskSerializer
    filter_class = TaskFilter
    search_fields = ('name', 'description', )
    ordering_fields = ('name', 'order', 'started', 'due', 'completed', )
        
class UserViewSet(DefaultsMixin, viewsets.ReadOnlyModelViewSet):
    """API endpoint for listing users."""
    
    lookup_field = User.USERNAME_FIELD
    lookup_url_kwarg = User.USERNAME_FIELD
    queryset = User.objects.order_by(User.USERNAME_FIELD)
    serializer_class = UserSerializer
    search_fields = (User.USERNAME_FIELD, )

Overwriting scrum/board/views.py


### Create links to link a sprint to its related tasks and users to their tasks

In [16]:
%%writefile scrum/board/serializers.py

from django.contrib.auth import get_user_model

from rest_framework import serializers
from rest_framework.reverse import reverse

from .models import Sprint, Task

User = get_user_model()

class SprintSerializer(serializers.ModelSerializer):
    
    links = serializers.SerializerMethodField()

    class Meta:
        model = Sprint
        fields = ('id', 'name', 'description', 'end', 'links', )

    def get_links(self, obj):
        request = self.context['request']
        return {
            'self': reverse('sprint-detail',
                kwargs={'pk': obj.pk},request=request),
            'tasks': reverse('task-list',
                request=request) + '?sprint={}'.format(obj.pk),
        }
    
class TaskSerializer(serializers.ModelSerializer):

    assigned = serializers.SlugRelatedField(
        slug_field=User.USERNAME_FIELD, required=False, queryset=User.objects.all())
    status_display = serializers.SerializerMethodField()
    links = serializers.SerializerMethodField()

    class Meta:
        model = Task
        fields = ('id', 'name', 'description', 'sprint',
            'status', 'status_display', 'order',
            'assigned', 'started', 'due', 'completed', 'links', )

    def get_status_display(self, obj):
        return obj.get_status_display()
    
    def get_links(self, obj):
        request = self.context['request']
        links = {
            'self': reverse('task-detail', 
                kwargs={'pk': obj.pk}, request=request),
            'sprint': None,
            'assigned': None
        }
        if obj.sprint_id:
            links['sprint'] = reverse('sprint-detail',
                kwargs={'pk': obj.sprint_id}, request=request)
        if obj.assigned:
            links['assigned'] = reverse('user-detail',
                kwargs={User.USERNAME_FIELD: obj.assigned}, request=request)
        return links

class UserSerializer(serializers.ModelSerializer):
    
    full_name = serializers.CharField(source='get_full_name', read_only=True)
    links = serializers.SerializerMethodField()

    class Meta:
        model = User
        fields = ('id', User.USERNAME_FIELD, 'full_name',
            'is_active', 'links', )

    def get_links(self, obj):
        request = self.context['request']
        username = obj.get_username()
        return {
            'self': reverse('user-detail',
                kwargs={User.USERNAME_FIELD: username}, request=request),
            'tasks': '{}?assigned={}'.format(
                reverse('task-list', request=request), username)
        }

Overwriting scrum/board/serializers.py


# Adding Validations
It allows changes to things that probably should not be changed.

1. It also makes it possible to create a sprint that has already ended. 
2. How the model data is serialized into a dictionary to later become JSON, XML, YAML, and so on for the client.
3. API should prevent is creating sprints that have happened prior to the current date and time

For a typical **Django view, this would be handled by a Form or ModelForm. In django-rest-framework, this is handled by the serializer.** 

**The API is similar to those of Django’s forms.** In fact, the serializer fields make use of the existing logic in Django’s form fields.

Each serializer field has a **validate_<field> hook that is called to perform additional validations on the field.** 